# Принцип работы YOLO

![image](Rn-V9RsehT8.jpg)

# Как происходит детекция
---
Для каждой клетки определяется класс и BB объекта в центре. На этапе инициализации сеть определяет размеры anchor box которые являются возможными вариантами Bounding box

![image](0f_-1wtt4qkwxr6bww4hpysnuwc.jpeg)

# Пример anchor box

![image](fxzqqy3rgqs_fh2s7ajninpcxos.jpeg)

# Выход каждой клетки
---
Для каждой клетки, нам нужно понять две принципиальные вещи:

1. Какой из anchor boxes, из 3 нарисованных вокруг клетки, нам подходит больше всего и как его можно немного подправить для того, чтобы он хорошо вписывал в себя объект
2. Какой объект находится внутри этого anchor box и есть ли он вообще

![image](q_6ehzuef4yaip17avcipq8wevg.png)

# Как определяется objectness?
---
 Этот параметр определяется с помощью метрики IoU во время обучения. Для нее определяется порог по которому bounding box будут исключены

# Архетиктура
---
В начале идет feature extraction и в конце стоит CNN слой который определяет что в клетках на которые было разделено изображение

# Пример архетиктуры

![image](bjldwmnqycbox81gunaedgon4go.jpeg)

# Слой детекции
---
Как мы видим из картинки, этот слой, размером 13x13 (для картинок изначального размера 416x416) для того, чтобы рассказывать про «каждую клетку» на картинке. Из этого последнего слоя и достается информация, которую мы хотим.

# YOLO предсказывает 5 параметров (для каждого anchor box для определенной клетки):
Задача YOLO — максимально точно предсказать эти параметры, чтобы максимально точно определять объект на картинке. А confidence score, который определяется для каждого предсказанного bounding box, является неким фильтром для того, чтобы отсеять совсем неточные предсказания. Для каждого предсказанного bounding box мы умножаем его IoU на вероятность того, что это определенный объект (вероятностное распределение рассчитывается во время обучения нейронной сети), берем лучшую вероятность из всех возможных, и если число после умножения превышает определенный порог, то мы можем оставить этот предсказанный bounding box на картинке.

![image](Screenshot2023-05-1845450.png)

# После отсеивания ложных предсказаний
---
Дальше, когда у нас остались только предсказанные bounding boxes с высоким confidence score, наши предсказания (если их визуализировать) могут выглядеть примерно вот так:

![image](we5nwbmeg9zm-jjuuh6hk79mkpa.png)

# NMS (non-max suppression)
---
Мы можем теперь использовать технику NMS (non-max suppression), чтобы отфильтровать bounding boxes таким образом, чтобы для одного объекта был только один предсказанный bounding box.
Процесс NMS выполняется после того, как алгоритм обнаружения вернул прямоугольные области (bounding boxes) и соответствующие им оценки уверенности (confidence scores) для каждого обнаруженного объекта. Цель NMS заключается в отсеве повторяющихся или перекрывающихся прямоугольных областей, оставляя только наиболее вероятные и уникальные объекты.

Основные шаги алгоритма NMS:

1. Сортировка всех обнаруженных прямоугольных областей по их оценкам уверенности (confidence scores) в порядке убывания.
2. Выбор области с наивысшей оценкой уверенности и добавление ее в окончательный список результатов. Эта область будет считаться "наилучшей" и будет служить представителем для других областей, которые с ней пересекаются.
3. Вычисление значений IoU (Intersection over Union) между наилучшей областью и всеми остальными областями в списке.
4. Удаление всех областей, у которых значение IoU выше определенного порога (обычно это значение между 0,5 и 0,7), поскольку они сильно перекрываются с наилучшей областью.
5. Повторение шагов 2-4 до тех пор, пока не будут обработаны все прямоугольные области в отсортированном списке.


Author: ChatGPT

![image](xpyyaqebpykn1y0h2_mgwbdaxum.png)

# Разные скейлы изображений в новых архетиктурах
---
YOLOv3-4 предсказывают на 3-х разных скейлах. То есть картинка делится на 64 grid cells, на 256 клеток и на 1024 клетки, чтобы также видеть маленькие объекты. Для каждой группы клеток алгоритм повторяет необходимые действия во время предсказания/обучения, которые были описаны сверху.

# Архетиктура YOLOv8
---
В данный момент соверменная архетиктура YOLOv8 использует свой собственный backbone и 3 скейла изображения

![image](232463267-9c83fb29-83da-4782-90ae-d142f9aa77d8.jpg)

# Использльваоние модели YOLOv8 для обучения и предсказания

In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="Codev.v3i.yolov8\\data.yaml", epochs=20,workers=4,device=0)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
results = model.predict(source='Codev.v3i.yolov8\\test\\images',save=True)
success = model.export(format="onnx")  # export the model to ONNX format

Ultralytics YOLOv8.0.104  Python-3.9.13 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\egors\PycharmProjects\object_detection\Codev.v3i.yolov8\data.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=